In [1]:
# importing Datasets and libraries 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# sep = \t tab separated file
data_yelp = pd.read_csv("yelp_labelled.txt",sep = "\t")

In [4]:
data_yelp.head()

# review and sentiment 
#0 - Negative , 1- Positive for postive review 

,Wow... Loved this place.,1
0,Crust is not good.,0
1,Not tasty and the texture was just nasty.,0
2,Stopped by during the late May bank holiday of...,1
3,The selection on the menu was great and so wer...,1
4,Now I am getting angry and I want my damn pho.,0


In [6]:
#Assign column names 

column_name = ['Review','Sentiment']
data_yelp.columns = column_name

In [7]:
data_yelp.head()

,Review,Sentiment
0,Crust is not good.,0
1,Not tasty and the texture was just nasty.,0
2,Stopped by during the late May bank holiday of...,1
3,The selection on the menu was great and so wer...,1
4,Now I am getting angry and I want my damn pho.,0


In [8]:
data_yelp.shape

(999, 2)

In [9]:
data_amazon =  pd.read_csv("amazon_cells_labelled.txt",sep="\t")

In [10]:
data_amazon.head()

,So there is no way for me to plug it in here in the US unless I go by a converter.,0
0,"Good case, Excellent value.",1
1,Great for the jawbone.,1
2,Tied to charger for conversations lasting more...,0
3,The mic is great.,1
4,I have to jiggle the plug to get it to line up...,0


In [11]:
data_amazon.columns = column_name

In [12]:
data_amazon.head()

,Review,Sentiment
0,"Good case, Excellent value.",1
1,Great for the jawbone.,1
2,Tied to charger for conversations lasting more...,0
3,The mic is great.,1
4,I have to jiggle the plug to get it to line up...,0


In [28]:
data_amazon.shape

(999, 2)

In [18]:
data_imdb = pd.read_csv("imdb_labelled.txt",sep="\t",header=None)

In [19]:
data_imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [24]:
data_imdb.columns = column_name

In [25]:
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [27]:
data_imdb.shape

(748, 2)

In [29]:
#Append all the data in a single data frame 

In [30]:
data = data_yelp.append([data_amazon,data_imdb],ignore_index = True)

In [31]:
data.shape

(2746, 2)

In [32]:
data.head()

,Review,Sentiment
0,Crust is not good.,0
1,Not tasty and the texture was just nasty.,0
2,Stopped by during the late May bank holiday of...,1
3,The selection on the menu was great and so wer...,1
4,Now I am getting angry and I want my damn pho.,0


In [33]:
# check distribution of sentiments 

In [34]:
data['Sentiment'].value_counts()

1    1385
0    1361
Name: Sentiment, dtype: int64

In [35]:
# check for null values 

data.isnull().sum()

Review       0
Sentiment    0
dtype: int64

In [37]:
# separate input and output dataset

x = data['Review']
y= data['Sentiment']

In [38]:
# Data Cleaning 
# Here we will remove stop words , punctuations
# as well as we will apply lemmatization 

In [39]:
# Creating function to clean the data 
import string
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [40]:
from spacy.lang.en.stop_words import STOP_WORDS

In [41]:
stopwords = list(STOP_WORDS)

In [45]:
import spacy
nlp= spacy.load("en_core_web_sm")

def text_data_cleaning(sentence):
  doc = nlp(sentence)

  tokens =[]
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)
  cleaned_tokens= []
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens

In [ ]:
# if root form of that word is not pronoun then it is going to cnvert that into lower form 
# and if that word is a proper noun, then we are direcly taking lower form , because there is no lemma 

In [46]:
text_data_cleaning("Hello all, It's a beautiful day outside there!")
# stop words and punctuation removed

['hello', 'beautiful', 'day', 'outside']

In [47]:
# Vectorization feature Engineering (TF-IDF)

In [48]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [51]:
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)

# tokenizer = text_data_cleaning , tokenization will be done according to this function 

In [52]:
classifier = LinearSVC()

In [54]:
# Train the model
# split the data into train and test

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size =0.2,random_state=0)

In [57]:
x_train.shape , x_test.shape

((2196,), (550,))

In [58]:
x_train.head()

2321    Watching washing machine twirling around would...
526     Be sure to order dessert, even if you need to ...
1509    You never know if you pushed it hard enough or...
144     The bathrooms are clean and the place itself i...
2482    Kris Kristoffersen is good in this movie and r...
Name: Review, dtype: object

In [60]:
# find the X_train and y_train

clf = Pipeline([('tfidf',tfidf),('clf',classifier)])

In [61]:
clf.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function text_data_cleaning at 0x7f991962abf8>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept

In [62]:
# in this we dont need to preapre the data set for testing 

In [64]:
# Predict the test result
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [65]:
y_pred = clf.predict(x_test)

In [66]:
confusion_matrix (y_test,y_pred)

array([[204,  62],
       [ 52, 232]])

In [68]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.80      0.77      0.78       266
           1       0.79      0.82      0.80       284

    accuracy                           0.79       550
   macro avg       0.79      0.79      0.79       550
weighted avg       0.79      0.79      0.79       550



In [69]:
accuracy_score(y_test,y_pred)

0.7927272727272727

In [70]:
clf.predict(["Wow, I am learning Natural language processing in fun fashion!"])
# op - 1 that means positive

array([1])

In [71]:
clf.predict(["It's hard to learn new things!"])

array([0])